In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 33 (delta 15), reused 26 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 8.20 KiB | 2.73 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/kaggle/working/agnews-classifier


In [54]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=2,
    lora_alpha=6,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 630,532 || all params: 125,279,240 || trainable%: 0.5033


In [55]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [56]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.366500,1.337776,0.735938
200,1.305200,1.253671,0.812500
300,1.167000,1.043076,0.864062
400,0.854500,0.639119,0.875000
500,0.552100,0.400524,0.890625
600,0.425300,0.346770,0.885938
700,0.379400,0.329139,0.895312
800,0.385500,0.318565,0.895312
900,0.355100,0.316868,0.898438
1000,0.355100,0.312125,0.898438


In [59]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:57<00:00,  4.37it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [60]:
import os
import pandas as pd

# Define paths
csv_path = "/kaggle/working/saved_models/checkpoint-2000/log_history.csv"
output_dir = "/kaggle/working/processed_data"
output_csv = os.path.join(output_dir, "processed_log_history.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the csv file
df = pd.read_csv(csv_path)

# Select desired columns and reorder the dataframe
desired_order = [
    "step",         
    "loss",         
    "eval_loss",   
    "accuracy",     
    "eval_accuracy",
    "learning_rate",
    "epoch"         
]
df = df[desired_order]

# Rename columns for uniformity
df.rename(columns={
    "step": "Step",
    "loss": "Train Loss",
    "eval_loss": "Test Loss",
    "accuracy": "Train Acc",
    "eval_accuracy": "Test Acc",
    "learning_rate": "Learning Rate",
    "epoch": "Epochs"
}, inplace=True)

# Adding loss spread and loss ratio columns
df["loss spread"] = df["Train Loss"] - df["Test Loss"]
df["loss ratio"] = df["Train Loss"] / df["Test Loss"]

# Adding acc spread and acc ratio columns
df["Acc spread"] = df["Test Acc"] - df["Train Acc"]
df["Acc ratio"] = df["Test Acc"] / df["Train Acc"]

# Export the DataFrame as a csv file
df.to_csv(output_csv, index=False)

print(f"Data exported to {output_csv}")

df

Data exported to /kaggle/working/processed_data/processed_log_history.csv


,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,loss spread,loss ratio,Acc spread,Acc ratio
0,100,1.3665,1.337776,0.376875,0.735938,0.000048,0.013405,0.028724,1.021471,0.359063,1.952736
1,200,1.3052,1.253671,0.597321,0.812500,0.000045,0.026810,0.051529,1.041103,0.215179,1.360239
2,300,1.1670,1.043076,0.777679,0.864062,0.000043,0.040214,0.123924,1.118807,0.086384,1.111079
3,400,0.8545,0.639119,0.869196,0.875000,0.000040,0.053619,0.215381,1.336997,0.005804,1.006677
4,500,0.5521,0.400524,0.884375,0.890625,0.000037,0.067024,0.151576,1.378446,0.006250,1.007067
5,600,0.4253,0.346770,0.887946,0.885938,0.000035,0.080429,0.078530,1.226462,-0.002009,0.997738
6,700,0.3794,0.329139,0.886161,0.895312,0.000033,0.093834,0.050261,1.152703,0.009152,1.010327
7,800,0.3855,0.318565,0.877232,0.895312,0.000030,0.107239,0.066935,1.210112,0.018080,1.020611
8,900,0.3551,0.316868,0.890625,0.898438,0.000028,0.120643,0.038232,1.120654,0.007812,1.008772
9,1000,0.3551,0.312125,0.889732,0.898438,0.000025,0.134048,0.042975,1.137684,0.008705,1.009784
